# Import

In [1]:
import os
import mne
from mne import events_from_annotations, create_info, EpochsArray, concatenate_epochs, Epochs
import json
import pandas as pd
from pathlib import Path
import re
import matplotlib.pyplot as plt
import numpy as np
import warnings

warnings.filterwarnings("ignore", message=".*boundary.*data discontinuities.*")
warnings.filterwarnings("ignore", message="FigureCanvasAgg is non-interactive, and thus cannot be shown")

# Classes

### EEGTaskData

In [2]:
class EEGTaskData:
    def __init__(self, subject, task, run, data_dir, l_freq=1, h_freq=40):
        self.subject = subject
        self.task = task
        self.run = run
        self.data_dir = data_dir
        self.l_freq = l_freq
        self.h_freq = h_freq

        self.raw = None
        self.metadata = {}
        self.events = None
        self.channels = None
        self.electrodes = None

        self.epochs = None
        self.label = None

        self._load()

        if task == 'RestingState':
            self._resting_preprocess()

    def _get_file(self, ext):
        name = f"sub-{self.subject}_task-{self.task}"
        if self.run:
            name += f"_run-{self.run}"
        return self.data_dir / f"{name}_{ext}"

    def _load(self):
        eeg_path = self._get_file("eeg.set")
        self.raw = mne.io.read_raw_eeglab(eeg_path, preload=True, montage_units='cm')
        montage = mne.channels.make_standard_montage("GSN-HydroCel-128")
        self.raw.drop_channels(['Cz'])
        self.raw.set_montage(montage, match_case=False)
        self.raw.filter(l_freq=self.l_freq, h_freq=self.h_freq)

        json_path = self._get_file("eeg.json")
        if json_path.exists():
            with open(json_path) as f:
                self.metadata = json.load(f)

        event_path = self._get_file("events.tsv")
        if event_path.exists():
            self.events = pd.read_csv(event_path, sep='\t')

        channels_path = self._get_file("channels.tsv")
        if channels_path.exists():
            self.channels = pd.read_csv(channels_path, sep='\t')

        electrodes_path = self._get_file("electrodes.tsv")
        if electrodes_path.exists():
            self.electrodes = pd.read_csv(electrodes_path, sep='\t')

    def _resting_preprocess(self, tmin=0.0, tmax=20.0):
        """
        Crop raw based on 'resting_start' to 'break cnt' in events.tsv,
        then epoch using eye condition annotations.
        """
        if self.events is None or self.raw is None:
            raise ValueError("Events or raw data not loaded.")

        # Step 1: Find resting_start and break cnt from TSV
        df = self.events

        t_start = df[df['value'] == 'resting_start']['onset'].values[0]
        t_end = df[df['value'] == 'break cnt']['onset'].values[1]

        # Step 2: Crop raw to this resting window
        self.raw.crop(tmin=t_start, tmax=t_end)

        # Step 3: Extract new events from cropped raw's annotations
        events, event_id = events_from_annotations(self.raw)

        eye_event_id = {
            'open': event_id['instructed_toOpenEyes'],
            'close': event_id['instructed_toCloseEyes']
        }

        # Step 4: Create epochs based on eye condition labels
        epochs = Epochs(
            self.raw,
            events=events,
            event_id=eye_event_id,
            tmin=tmin,
            tmax=tmax,
            proj=True,
            baseline=None,
            preload=True
        )

        self.epochs = epochs
        self.labels = self.epochs.events[:, -1] - eye_event_id['open']  # 0=open, 1=close


    def show_annotations(self):
        return self.metadata if self.metadata else None

    def show_table(self, name='events', rows=10):
        df_map = {
            'events': self.events,
            'channels': self.channels,
            'electrodes': self.electrodes
        }

        if name == 'epochs' and self.epochs is not None:
            info = {
                'n_epochs': len(self.epochs),
                'n_channels': len(self.epochs.ch_names),
                'timespan_sec': self.epochs.times[-1] - self.epochs.times[0],
                'labels': np.unique(self.labels) if self.labels is not None else 'N/A',
                'sampling_rate': self.epochs.info['sfreq'],
                'duration_per_epoch_sec': self.epochs.get_data().shape[-1] / self.epochs.info['sfreq']
            }
            return pd.DataFrame([info])

        df = df_map.get(name)
        return df.head(rows) if df is not None else None
    
    def get_raw(self):
        return self.raw
    
    def get_eye_epochs(self):
        """Returns (epochs, labels) if available, else (None, None)."""
        return self.epochs, self.labels


### EEGSubjectData

In [3]:
class EEGSubjectData:
    def __init__(self, data_directory, l_freq=None, h_freq=50):
        self.data_dir = Path(data_directory)
        self.subjects = {}  # subject_id → { (task, run): EEGTaskData }
        self.l_freq = l_freq
        self.h_freq = h_freq

        self._load_subjects()

    def _load_subjects(self):
        pattern = re.compile(r"sub-(?P<subject>[^_]+)_task-(?P<task>[^_]+)(?:_run-(?P<run>\d+))?_eeg.set")
        for file in self.data_dir.glob("sub-*_task-*_eeg.set"):
            match = pattern.match(file.name)
            if match:
                subject = match.group("subject")
                task = match.group("task")
                run = match.group("run")

                if subject not in self.subjects:
                    self.subjects[subject] = {}

                key = (task, run)
                self.subjects[subject][key] = EEGTaskData(subject, task, run, self.data_dir, self.l_freq, self.h_freq)

    def get_task(self, subject, task, run=None):
        return self.subjects.get(subject, {}).get((task, run))


### EEGVisualization

In [4]:
class EEGVisualization:
    def __init__(self, subject_data: EEGSubjectData):
        self.data = subject_data

    def plot_sensors(self, subject, task, run=None):
        task_data = self.data.get_task(subject, task, run)
        if task_data and task_data.raw:
            task_data.raw.plot_sensors(show_names=True)

    def plot_time(self, subject, task, run=None, **kwargs):
        task_data = self.data.get_task(subject, task, run)
        raw = task_data.raw

        duration = kwargs.get('duration', 10.0)
        start = kwargs.get('start', 0.0)
        n_channels = kwargs.get('n_channels', 10)

        title = f"{subject} - {task}" + (f" (Run {run})" if run else "")
        raw.plot(
            title=title,
            duration=duration,
            start=start,
            n_channels=n_channels,
            scalings='auto',
            show=True,
            block=True
        )

    def plot_frequency(self, subject, task, run=None, **kwargs):
        task_data = self.data.get_task(subject, task, run)
        raw = task_data.raw

        fmin = kwargs.get("fmin", 1)
        fmax = kwargs.get("fmax", 60)
        average = kwargs.get("average", True)
        dB = kwargs.get("dB", True)
        spatial_colors = kwargs.get("spatial_colors", False)
        show = kwargs.get("show", True)

        psd = raw.compute_psd(fmin=fmin, fmax=fmax)
        psd.plot(average=average, spatial_colors=spatial_colors, dB=dB, show=show)

    def plot_conditionwise_psd(self, subject, task, run=None, **kwargs):
        fmin = kwargs.get("fmin", 1)
        fmax = kwargs.get("fmax", 50)
        tmin = kwargs.get("tmin", None)
        tmax = kwargs.get("tmax", None)
        average = kwargs.get("average", True)
        dB = kwargs.get("dB", True)

        task_data = self.data.get_task(subject, task, run)
        epochs = task_data.epochs
        event_ids = epochs.event_id

        for condition_name in event_ids:
            condition_epochs = epochs[condition_name]
            if tmin is not None or tmax is not None:
                condition_epochs = condition_epochs.copy().crop(tmin=tmin, tmax=tmax)

            psd = condition_epochs.compute_psd(fmin=fmin, fmax=fmax)
            psd.plot(spatial_colors=True, average=average, dB=dB)
            fig = plt.gcf()
            fig.suptitle(f"{subject} - {task} - {condition_name}", fontsize=14)
            fig.subplots_adjust(top=0.85)

            if tmin is not None or tmax is not None:
                caption = f"Epoch time window: tmin = {tmin if tmin is not None else 'auto'}, tmax = {tmax if tmax is not None else 'auto'}"
                fig.text(0.5, 0.01, caption, ha='center', fontsize=10)


### EEGController

In [5]:
class EEGController:
    def __init__(self, subject_data: 'EEGSubjectData', visualizer: 'EEGVisualization'):
        self.subject_data = subject_data
        self.visualizer = visualizer

    def list_subjects(self):
        return list(self.subject_data.subjects.keys())

    def list_tasks(self, subject):
        return list(self.subject_data.subjects.get(subject, {}).keys())

    def show(self, subject, task, run=None, plot_type='time', **kwargs):
        if plot_type == 'time':
            self.visualizer.plot_time(subject, task, run, **kwargs)
        elif plot_type == 'sensors':
            self.visualizer.plot_sensors(subject, task, run)
        elif plot_type == 'frequency':
            self.visualizer.plot_frequency(subject, task, run)
        elif plot_type == 'conditionwise psd':
            self.visualizer.plot_conditionwise_psd(subject, task, run, **kwargs)

    def show_annotations(self, subject, task, run=None):
        """Return metadata dict or None."""
        task_data = self.subject_data.get_task(subject, task, run)
        return task_data.show_annotations() if task_data else None

    def show_table(self, subject, task, run=None, name='events', rows=10):
        """Return DataFrame or None."""
        task_data = self.subject_data.get_task(subject, task, run)
        return task_data.show_table(name=name, rows=rows) if task_data else None

    def get_annotation_df(self, subject, task, run=None):
        task_data = self.subject_data.get_task(subject, task, run)

        raw = task_data.get_raw()
        
        annots = raw.annotations
        df = pd.DataFrame({
            "onset": annots.onset,
            "duration": annots.duration,
            "description": annots.description
        })
        return df

### EEGUI

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json  # Needed for metadata display

class EEGUI:
    def __init__(self, controller: 'EEGController'):
        self.controller = controller

        # Get and sort subject list
        subjects = sorted(self.controller.list_subjects())

        # Widgets
        self.subject_dropdown = widgets.Dropdown(
            options=subjects,
            description='Subject:',
            layout=widgets.Layout(width='250px')
        )

        self.task_dropdown = widgets.Dropdown(
            options=[], 
            description='Task:',
            layout=widgets.Layout(width='250px')
        )

        self.run_text = widgets.Text(
            description='Run:',
            placeholder='Optional',
            layout=widgets.Layout(width='250px')
        )

        self.plot_type = widgets.ToggleButtons(
            options=['time', 'sensors', 'frequency', 'conditionwise psd'],
            description='Plot:',
            layout=widgets.Layout(width='600px')
        )

        # Frequency Domain
        self.average_check = widgets.Checkbox(
            value=True,
            description='Average',
            indent=False
        )

        self.db_check = widgets.Checkbox(
            value=True,
            description='dB',
            indent=False
        )

        # Time Domain
        self.duration_float = widgets.FloatText(value=10.0, description='duration:', layout=widgets.Layout(width='200px'))
        self.start_float = widgets.FloatText(value=0.0, description='start:', layout=widgets.Layout(width='200px'))
        self.nchan_int = widgets.IntText(value=10, description='n_channels:', layout=widgets.Layout(width='200px'))

        # Time and frequency inputs
        self.tmin_float = widgets.FloatText(value=0.0, description='tmin:', layout=widgets.Layout(width='200px'))
        self.tmax_float = widgets.FloatText(value=2.0, description='tmax:', layout=widgets.Layout(width='200px'))
        self.fmin_float = widgets.FloatText(value=1.0, description='fmin:', layout=widgets.Layout(width='200px'))
        self.fmax_float = widgets.FloatText(value=50.0, description='fmax:', layout=widgets.Layout(width='200px'))

        # Param containers
        self.t_controls = widgets.HBox([self.tmin_float, self.tmax_float])
        self.f_controls = widgets.HBox([self.fmin_float, self.fmax_float])
        self.time_controls = widgets.HBox([self.duration_float, self.start_float, self.nchan_int])
        self.psd_options = widgets.HBox([self.average_check, self.db_check])
        self.param_box = widgets.VBox([])

        self.plot_button = widgets.Button(description='Plot', button_style='success')

        self.table_type = widgets.Dropdown(
            options=['events', 'channels', 'electrodes', 'epochs'],
            description='Table:',
            layout=widgets.Layout(width='250px')
        )

        self.info_button = widgets.Button(description='Show Info', button_style='info')
        self.output = widgets.Output()

        # Link events
        self.subject_dropdown.observe(self.update_tasks, names='value')
        self.plot_type.observe(self.update_param_inputs, names='value')
        self.plot_button.on_click(self.do_plot)
        self.info_button.on_click(self.do_show_info)

        # UI layout
        self.ui = widgets.VBox([
            self.subject_dropdown,
            self.task_dropdown,
            self.run_text,
            self.plot_type,
            self.param_box,
            self.plot_button,
            widgets.HBox([self.table_type, self.info_button]),
            self.output
        ])

        # Init values
        if subjects:
            self.subject_dropdown.value = subjects[0]
            self.update_tasks()
        self.update_param_inputs()  # Show/hide params based on default selection

    def update_tasks(self, *args):
        subject = self.subject_dropdown.value
        task_keys = sorted(self.controller.list_tasks(subject))
        formatted = [(f"{t} (run {r})" if r else t, (t, r)) for t, r in task_keys]
        self.task_dropdown.options = formatted
        if formatted:
            self.task_dropdown.value = formatted[0][1]

    def update_param_inputs(self, *args):
        if self.plot_type.value == 'conditionwise psd':
            self.param_box.children = [self.t_controls, self.f_controls, self.psd_options]
        elif self.plot_type.value == 'frequency':
            self.param_box.children = [self.f_controls, self.psd_options]
        elif self.plot_type.value == 'time':
            self.param_box.children = [self.time_controls]
        else:
            self.param_box.children = []


    def do_plot(self, _):
        with self.output:
            clear_output(wait=True)

            subject = self.subject_dropdown.value
            task_value = self.task_dropdown.value
            if task_value is None:
                print("No task selected.")
                return

            task, run = task_value
            run = run if run else None

            kwargs = {
                'tmin': self.tmin_float.value,
                'tmax': self.tmax_float.value,
                'fmin': self.fmin_float.value,
                'fmax': self.fmax_float.value,
                'duration': self.duration_float.value,
                'start': self.start_float.value,
                'n_channels': self.nchan_int.value,
                'average': self.average_check.value,
                'dB': self.db_check.value
            }

            self.controller.show(subject, task, run, plot_type=self.plot_type.value, **kwargs)

    def do_show_info(self, _):
        with self.output:
            clear_output(wait=True)
            subject = self.subject_dropdown.value
            task_value = self.task_dropdown.value
            if task_value is None:
                print("No task selected.")
                return

            task, run = task_value
            run = run if run else None

            metadata = self.controller.show_annotations(subject, task, run)
            print(f"Metadata for {subject} - {task} (Run {run}):")
            if metadata:
                print(json.dumps(metadata, indent=2))
            else:
                print("No metadata available.")

            table_name = self.table_type.value
            df = self.controller.show_table(subject, task, run, name=table_name)
            print(f"\nTable: {table_name}")
            if df is not None:
                display(df)
            else:
                print("No table data available.")

    def show(self):
        display(self.ui)

# Initialize

### Load

In [7]:
load = 1
if load:
    data_dir = '/mount/sub/cmi_bids_R1/eeg'
    subject_data = EEGSubjectData(data_dir, l_freq=1, h_freq=50)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    9.1s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    4.6s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.5s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.7s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s


### UI

In [8]:
%matplotlib inline

visualizer = EEGVisualization(subject_data)
controller = EEGController(subject_data, visualizer)

ui = EEGUI(controller)
ui.show()

In [9]:
task_data = controller.subject_data.get_task("NDARAC904DMU","RestingState")
epochs = task_data.epochs
epochs.event_id

{'open': 3, 'close': 2}